In [1]:
import pandas as pd

mini_encounter_vector = pd.read_csv('/data/MIMIC/mini_encounter_vectors.csv')
encounter_vector = pd.read_csv('/data/MIMIC/encounter_vectors.csv', na_values='\N')
print(encounter_vector.shape)
encounter_vector.columns = mini_encounter_vector.columns

/home/brett/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/brett/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5,8,9,602) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1566025, 935)


In [2]:
print(mini_encounter_vector.head())
print(encounter_vector.head())

   ENCOUNTER_ID  HADM_ID  SUBJECT_ID            ADMITTIME  \
0             1   100001       58526  2117-09-11 11:46:00   
1             2   100001       58526  2117-09-11 11:46:00   
2             3   100001       58526  2117-09-11 11:46:00   
3             4   100001       58526  2117-09-11 11:46:00   
4             5   100001       58526  2117-09-11 11:46:00   

             DISCHTIME DEATHTIME                  DOB  DOD DOD_HOSP DOD_SSN  \
0  2117-09-17 16:45:00       NaN  2082-03-21 00:00:00  NaN      NaN     NaN   
1  2117-09-17 16:45:00       NaN  2082-03-21 00:00:00  NaN      NaN     NaN   
2  2117-09-17 16:45:00       NaN  2082-03-21 00:00:00  NaN      NaN     NaN   
3  2117-09-17 16:45:00       NaN  2082-03-21 00:00:00  NaN      NaN     NaN   
4  2117-09-17 16:45:00       NaN  2082-03-21 00:00:00  NaN      NaN     NaN   

                ...               Resp Rate_STD SaO2 Alarm [Low]_STD  \
0               ...                         NaN                  NaN   
1             

In [3]:
# copying so can run multiple times for testing with out breaking
e_vect = encounter_vector
e_vect.drop([col for col in list(e_vect) if col.startswith('HADM_ID.')],
             axis=1,
             inplace=True)
e_vect.drop([col for col in list(e_vect) if col.startswith('ENCOUNTER_ID.')],
             axis=1,
             inplace=True)
print(e_vect.shape)

(1566025, 911)


In [4]:
i = 0
for col in e_vect:
    print(i, col, len(e_vect[col].unique()))
    i += 1

(0, 'ENCOUNTER_ID', 1566025)
(1, 'HADM_ID', 58438)
(2, 'SUBJECT_ID', 46297)
(3, 'ADMITTIME', 58122)
(4, 'DISCHTIME', 58127)
(5, 'DEATHTIME', 5823)
(6, 'DOB', 32413)
(7, 'DOD', 12901)
(8, 'DOD_HOSP', 8739)
(9, 'DOD_SSN', 11293)
(10, 'ADMISSION_TYPE', 4)
(11, 'ADMISSION_LOCATION', 9)
(12, 'DISCHARGE_LOCATION', 17)
(13, 'GENDER', 2)
(14, 'INSURANCE', 5)
(15, 'RELIGION', 21)
(16, 'LANGUAGE', 76)
(17, 'MARITAL_STATUS', 8)
(18, 'ETHNICITY', 41)
(19, 'EDREGTIME', 30856)
(20, 'EDOUTTIME', 30846)
(21, 'DIAGNOSIS', 15578)
(22, 'Equip Change [MM]', 119)
(23, 'INV#1 Dsg Change [MM]', 124)
(24, 'INV#2 Dsg Change [MM]', 113)
(25, 'INV#3 Dsg Change [MM]', 109)
(26, 'INV#4 Dsg Change [MM]', 33)
(27, 'INV#5 Dsg Change [MM]', 53)
(28, 'INV#6 Dsg Change [MM]', 113)
(29, 'INV#7 Dsg Change [MM]', 53)
(30, 'INV#1 Cap Change [MM]', 238)
(31, 'INV#1 Tubing Change [MM]', 113)
(32, 'INV#2 Cap Change [MM]', 53)
(33, 'INV#2 Tubing Change [MM]', 50)
(34, 'INV#3 Cap Change [MM]', 48)
(35, 'INV#3 Tubing Change [MM]'

In [5]:
#print(pd.get_dummies(e_vect['Equip Change [MM]']))
# print(e_vect['Equip Change [MM]'].value_counts())
print(e_vect['DOD'].head())


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: DOD, dtype: object


In [6]:
# INDEX - ENCOUNTER_ID, HADM_ID, SUBJECT_ID
index_lists = [e_vect['SUBJECT_ID'].tolist(),
                e_vect['HADM_ID'].tolist(),
                e_vect['ENCOUNTER_ID'].tolist()]

tuples = list(zip(*index_lists))
index = pd.MultiIndex.from_tuples(tuples, 
                                  names=['SUBJECT_ID',
                                         'HADM_ID',
                                         'ENCOUNTER_ID'])

encounters = pd.DataFrame()

# LABELS - DOD
encounters['SURVIVAL'] = (pd.to_datetime(e_vect['DOD']) -
                          pd.to_datetime(e_vect['ADMITTIME'])).dt.days

# Dates - ADMITTIME, DISCHTIME, DEATHTIME, DOB, DOD, DOD_HOSP, DOD_SSN
encounters['AGE'] = (pd.to_datetime(e_vect['ADMITTIME']) - 
                     pd.to_datetime(e_vect['DOB'])).dt.days 
                                                                  

# First handle static demographic data (mostly cateogrical)
encounters = pd.concat([encounters,
                        pd.get_dummies(e_vect['GENDER'])['M'],
                        pd.get_dummies(e_vect['ADMISSION_TYPE']),
                        pd.get_dummies(e_vect['ADMISSION_LOCATION']),
                        pd.get_dummies(e_vect['INSURANCE']),
                        pd.get_dummies(e_vect['RELIGION']),
                        pd.get_dummies(e_vect['LANGUAGE']),
                        pd.get_dummies(e_vect['MARITAL_STATUS']),
                        pd.get_dummies(e_vect['ETHNICITY'])
                       ],
                       axis=1)

# DTE Table - important to remember that end is not inclusive
encounters = pd.concat([encounters, e_vect.ix[:, 22:150]], axis=1)
# ICU Table
encounters = pd.concat([encounters, e_vect['ICUSTART'], e_vect['ICUEND']], axis=1)
# INPUT CV
encounters = pd.concat([encounters, e_vect.ix[:, 152:279]], axis=1)
# INPUT MV
encounters = pd.concat([encounters, e_vect.ix[:, 279:339]], axis=1)
# Lab Events
encounters = pd.concat([encounters, e_vect.ix[:, 339:476]], axis=1)
# Procedure Events
encounters = pd.concat([encounters, e_vect.ix[:, 476:543]], axis=1)
# Microbiology Events
encounters = pd.concat([encounters, e_vect.ix[:, 566:587]], axis=1)
# Services
encounters = pd.concat([encounters, pd.get_dummies(e_vect['SERVICE'])], axis=1)
# Note Events
encounters = pd.concat([encounters, e_vect.ix[:, 587:601]], axis=1)
# Output Events
encounters = pd.concat([encounters, e_vect.ix[:, 602:635]], axis=1)
# Prescription Events
encounters = pd.concat([encounters, e_vect.ix[:, 635:767]], axis=1)
# STREAMING CE
encounters = pd.concat([encounters, e_vect.ix[:, 767:]], axis=1)

encounters.set_index(index, inplace=True)
print(encounters.shape)
print(encounters.head())

/home/brett/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


(1566025, 1089)
                                 SURVIVAL    AGE  M  ELECTIVE  EMERGENCY  \
SUBJECT_ID HADM_ID ENCOUNTER_ID                                            
58526      100001  2                  NaN  12957  0         0          1   
                   3                  NaN  12957  0         0          1   
                   4                  NaN  12957  0         0          1   
                   5                  NaN  12957  0         0          1   
                   6                  NaN  12957  0         0          1   

                                 NEWBORN  URGENT  ** INFO NOT AVAILABLE **  \
SUBJECT_ID HADM_ID ENCOUNTER_ID                                              
58526      100001  2                   0       0                         0   
                   3                   0       0                         0   
                   4                   0       0                         0   
                   5                   0       0             

In [7]:
# let's handle null now
nan_cols = encounters.columns[encounters.isnull().any()].tolist()

for col in nan_cols:
    print(col)
    # first make column for presence
    encounters[col + '_present'] = encounters[col].notnull().astype(int)
    
    # then fill na with 0
    encounters[col]=encounters[col].fillna(0)

SURVIVAL
Arterial BP [Systolic]_MAX
Arterial BP Mean_MAX
Ectopy Frequency_MAX
Ectopy Type_MAX
HEART Rate_MAX
Heart Rhythm_MAX
NBP [Systolic]_MAX
NBP Mean_MAX
Precautions_MAX
Previous WeightF_MAX
Respiratory Rate_MAX
SpO2_MAX
HR Alarm [Low]_MAX
SaO2_MAX
NBP Alarm [Low]_MAX
Resp Alarm [Low]_MAX
SpO2 Alarm [Low]_MAX
Resp Rate_MAX
SaO2 Alarm [Low]_MAX
Arterial BP [Diastolic]_MAX
NBP [Diastolic]_MAX
HR Alarm [High]_MAX
NBP Alarm [High]_MAX
Resp Alarm [High]_MAX
SpO2 Alarm [High]_MAX
SaO2 Alarm [High]_MAX
O2 saturation pulseoxymetry_MAX
Arterial BP [Systolic]_MIN
Arterial BP Mean_MIN
Ectopy Frequency_MIN
Ectopy Type_MIN
HEART Rate_MIN
Heart Rhythm_MIN
NBP [Systolic]_MIN
NBP Mean_MIN
Precautions_MIN
Previous WeightF_MIN
Respiratory Rate_MIN
SpO2_MIN
HR Alarm [Low]_MIN
SaO2_MIN
NBP Alarm [Low]_MIN
Resp Alarm [Low]_MIN
SpO2 Alarm [Low]_MIN
Resp Rate_MIN
SaO2 Alarm [Low]_MIN
Arterial BP [Diastolic]_MIN
NBP [Diastolic]_MIN
HR Alarm [High]_MIN
NBP Alarm [High]_MIN
Resp Alarm [High]_MIN
SpO2 Alarm 

In [14]:
encounters.select_dtypes(include=['object']).columns
encounters.drop(['70023_i'], axis=1, inplace=True)

In [15]:
encounters.select_dtypes(include=['object']).columns

Index([], dtype='object')

In [16]:
encounters.to_csv('/data/MIMIC/encounter_vectors_processed.csv')